In [3]:
import torch
from torch import nn
import numpy as np

/home/student/anaconda3/envs/nlp/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
k=3
vocab_size=10

In [8]:
word_history = [[0], [4], [2]] # [k, |history|]

# Pruning

In [5]:
    # once satisfied remove
    pos_constraints = [[[1, 4], [5], [7, 9], [0]] for _ in range(k)]
    # no need to remove
    neg_constraints = [[3], [6], [2, 1]]

In [7]:
decoder_output = torch.rand(k, vocab_size)
decoder_output

tensor([[0.2842, 0.6313, 0.1902, 0.0574, 0.1085, 0.6113, 0.8863, 0.4447, 0.2815,
         0.6691],
        [0.7425, 0.2465, 0.0646, 0.7713, 0.4118, 0.7276, 0.1250, 0.5677, 0.0539,
         0.0690],
        [0.2149, 0.3084, 0.9116, 0.7402, 0.5656, 0.6229, 0.4891, 0.5549, 0.6596,
         0.6171]])

### penalize inclusion of negative constraint

In [15]:
neg_constraint_penalty = 1

In [14]:
penalties = torch.zeros_like(decoder_output) # [k, vocab_size]

In [16]:
for ki in range(k):
    for neg_constraint in neg_constraints:
        neg_constraint_exist = True
        neg_idx = neg_constraint[-1] # index in vocab dim of the negative word [3, 6, 1]
        for i, constraint_word in enumerate(neg_constraint[:-1][::-1]): # from 2nd last to first word in constraint
            if word_history[ki][-i] != constraint_word:
                neg_constraint_exist = False
                break
        penalties[ki][neg_idx] += neg_constraint_penalty

# Selection